In [22]:
import json
import pandas as pd

In [59]:
docs = pd.read_csv('data/annotate_text.csv')

In [60]:
docs.head()

,row_id,text
0,607526,Chief Complaint:\n 24 Hour Events:\n PEEP ...
1,685923,The patient is a 28 year old male with a histo...
2,513639,"Gastrointestinal bleed, other (GI Bleed, GIB)\..."
3,581559,"Chief Complaint: hypotension, CHF\n I saw an..."
4,685928,Chief Complaint:\n 24 Hour Events:\n MULTI L...


In [61]:
data = json.load(open('data/MedCAT_Export_With_Text_2021-05-10_10_28_34.json'))

In [75]:
# data['projects'][0]['documents']

In [63]:
len(data['projects'][0]['documents'])

50

In [77]:
proj['documents'][0]

{'id': 1,
 'name': 'Doc 0',
 'text': 'Chief Complaint:\n   24 Hour Events:\n   PEEP and FIO2 weaned overnight\n   Allergies:\n   No Known Drug Allergies\n   Last dose of Antibiotics:\n   Ceftriaxone - [**2181-10-26**] 09:57 AM\n   Vancomycin - [**2181-10-27**] 08:00 PM\n   Ciprofloxacin - [**2181-10-28**] 12:00 AM\n   Piperacillin/Tazobactam (Zosyn) - [**2181-10-28**] 12:00 AM\n   Infusions:\n   Propofol - 7 mcg/Kg/min\n   Midazolam (Versed) - 1.5 mg/hour\n   Fentanyl (Concentrate) - 50 mcg/hour\n   Octreotide - 50 mcg/hour\n   Other ICU medications:\n   Other medications:\n   Changes to medical and family history:\n   Review of systems is unchanged from admission except as noted below\n   Review of systems:\n   Flowsheet Data as of  [**2181-10-28**] 07:41 AM\n   Vital signs\n   Hemodynamic monitoring\n   Fluid balance\n                                                                  24 hours\n                                                               Since 12 AM\n   Tmax: 37.3\nC

In [65]:
proj = data['projects'][0]

In [78]:
annos = []

for doc in proj['documents']:
    did = doc['name']

    text = doc['text']
    for a in doc['annotations']:
        if a['validated']:
            user = a['user']
#             aid = a['id']
            aid = str(a['start']) + '-' + str(a['end'])
            value = a['value']
#                 context = text[a['start']-200:a['end']+200:]
            context = text[a['start']-200: a['start']] + ' ******Start****** ' + value + ' ******End****** ' + text[a['end']: a['end']+200]
#             print(user)
            cui = a['cui']
            correct = a['correct']
            meta_anns = a['meta_anns']
            dia = ''
            state = ''
            if len(meta_anns) != 0:
                if 'Negated' in meta_anns:
                    dia = meta_anns['Negated']['value']
                if 'Skip' in meta_anns: 
                    state = meta_anns['Skip']['value']
            annos.append([did, aid, value, context, cui, user, correct, dia, state, str(correct) + ' | ' + dia + ' | '+ state])

In [91]:
df = pd.DataFrame(annos, columns=['doc_id', 'annotation_id', 'string', 'context', 'cui', 'user', 'correct', 'Negated', 'Skip', 'combine'])

In [92]:
df.describe()

,doc_id,annotation_id,string,context,cui,user,correct,Negated,Skip,combine
count,9953,9953,9953,9953,9953,9953,9953,9953,9953,9953
unique,50,9353,3650,9950,2292,2,2,4,3,13
top,Doc 45,0-15,Assessment,******Start****** saw ******End****** and ex...,C1261322,helena,True,No,No,True | No | No
freq,569,12,89,3,95,9656,8543,8840,8901,7900


In [93]:
df['Negated'].unique()

array(['', 'No', 'Yes', 'Unknown'], dtype=object)

In [94]:
df['Skip'].unique()

array(['', 'No', 'Yes'], dtype=object)

In [95]:
df[df['user']=='tao'].describe()

,doc_id,annotation_id,string,context,cui,user,correct,Negated,Skip,combine
count,297,297,297,297,297,297,297,297,297,297
unique,11,292,172,297,158,1,2,4,3,8
top,Doc 3,2496-2503,Doctor,SE-SUCTIONED FOR SMALL AMT YELLOW/BLOOD TINGED...,C0237753,tao,True,No,No,True | No | No
freq,52,2,7,1,7,297,175,194,189,164


In [96]:
df[df['user']=='helena'].describe()

,doc_id,annotation_id,string,context,cui,user,correct,Negated,Skip,combine
count,9656,9656,9656,9656,9656,9656,9656,9656,9656,9656
unique,50,9100,3590,9654,2267,1,2,4,3,13
top,Doc 45,0-15,Assessment,******Start****** examined ******End****** t...,C1261322,helena,True,No,No,True | No | No
freq,569,12,85,2,91,9656,8368,8646,8712,7736


In [97]:
df = df[df['user']=='helena']

In [98]:
df.shape

(9656, 10)

In [101]:
df['correct'].unique()

array([ True, False])

In [104]:
df[df['correct']==False].head()

,doc_id,annotation_id,string,context,cui,user,correct,Negated,Skip,combine
92,Doc 0,2815-2823,Movement,"ht radial pulse: Not assessed), (Left radial\n...",C0026649,helena,False,,,False | |
98,Doc 0,53-59,weaned,******Start****** weaned ******End****** ove...,C0043084,helena,False,,,False | |
110,Doc 0,3908-3913,Lymph,22\n 21\n 22\n 20\n 21\n Glucose\n...,C0024202,helena,False,,,False | |
117,Doc 0,923-927,Tmax,gns\n Hemodynamic monitoring\n Fluid balan...,C0039476,helena,False,No,No,False | No | No
129,Doc 0,3840-3844,Bili,9.7\n 39.2\n 38.7\n Plt\n 74\n 81\n ...,C0520687,helena,False,,,False | |


## Re-annoate data

In [109]:
from medcat.cat import CAT
from medcat.utils.vocab import Vocab
from medcat.cdb import CDB
from medcat.meta_cat import MetaCAT

In [110]:
cdb = CDB()
    # cdb.load_dict("/home/ubuntu/Tao/KER/medcat_models/0.2.6.2 - snomed_us_ext_names_umls_clean_primary_1M.dat")
cdb.load_dict("medcat_models/umls_base_wlink_clean_name_400k_mimic.dat")
vocab = Vocab()
vocab.load_dict(path='medcat_models/base_vocabulary.dat')


mc_negated = MetaCAT(save_dir="medcat_models/mc_negated/")
mc_negated.load()
mc_skip = MetaCAT(save_dir="medcat_models/mc_skip/")
mc_skip.load()


cat = CAT(cdb, vocab=vocab, meta_cats=[mc_negated, mc_skip])
cat.train = False
cat.spacy_cat.MIN_ACC = 0.35
cat.spacy_cat.MIN_CONCEPT_LENGTH = 2

ModuleNotFoundError: No module named 'thinc.types'